In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = 'C:/Users/User/Desktop/Desktop/SIGNATURE_v2.0/data/train'
#TEST_DIR = 'C:/Users/User/Desktop/Desktop/SIGNATURE_v2.0/data/test'
IMG_SIZE = 100
LR = 9e-6

MODEL_NAME = 'signatures-{}-{}.model'.format(LR, '2conv')

In [2]:
def label_img(img):
    word_label = img.split('.')[0]
    if word_label == 'Ivanov': return[1,0]
    elif word_label == 'Sergeev': return [0,1]

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [4]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    np.save('test_data.npy', testing_data)
    return testing_data

In [5]:
train_data = create_train_data()
# if training data is already exist:
#train_data = np.load('train_data.npy')

100%|███████████████████████████████████████████████████████████████| 594/594 [00:02<00:00, 220.49it/s]


In [6]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
tf.reset_default_graph()
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet)

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [7]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
        model.load(MODEL_NAME)
        print('model loaded')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\User\Desktop\Desktop\SIGNATURE_v2.0\signatures-9e-06-2conv.model
model loaded


In [8]:
train = train_data[:-100]
test = train_data[-100:]

In [9]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

In [10]:
model.fit({'input': X}, {'targets': Y}, n_epoch=8, validation_set=({'input': test_x}, {'targets': test_y}), show_metric=True, run_id=MODEL_NAME)

Training Step: 127  | total loss: 0.44588 | time: 0.386s
| Adam | epoch: 008 | loss: 0.44588 - acc: 0.7867 -- iter: 448/494
Training Step: 128  | total loss: 0.44873 | time: 1.454s
| Adam | epoch: 008 | loss: 0.44873 - acc: 0.7841 | val_loss: 0.29497 - val_acc: 0.9500 -- iter: 494/494
--


In [11]:
model.save(MODEL_NAME)

INFO:tensorflow:C:\Users\User\Desktop\Desktop\SIGNATURE_v2.0\signatures-9e-06-2conv.model is not in all_model_checkpoint_paths. Manually adding it.
